In [1]:
import hashlib
import rsa
import os
from gmpy2 import mpz, iroot   # For taking the ith root of large numbers

In [2]:
def to_bytes(n):
    return n.to_bytes((n.bit_length() // 8) + 1, byteorder='big')
    
def from_bytes(n):
    return int.from_bytes(n,byteorder='big')

def get_bit(n, b):
    # Returns bth bit from the least significant bit starting from zero
    return ((1 << b) & n) >> b

def set_bit(n,b):
    # Returns the number after changing bth bit to 1 from least significant bit
    return (1<<b) | n

def cube_root(n):
    # Returns ith Root of Large Numbers
    return int(iroot(mpz(n),3)[0])

In [3]:
cube_root(8)

2

In [4]:
message = "My name is Shreyansh!".encode("ASCII")
print(message)
message_hash = hashlib.sha256(message).digest()
print(message_hash)

b'My name is Shreyansh!'
b'CWS\xd9\xa6\xb1\x0b\xfa)L\xd7\xee\x95\x0b\xa6\x0e6pu\xa5\xdbS\x0b\x12\x9d\x17\xf5\xd1\n\xd1\xf1{'


In [117]:
ASN1_blob = rsa.pkcs1.HASH_ASN1['SHA-256']
print(ASN1_blob.hex())
suffix = b'\x00' + ASN1_blob + message_hash
print(suffix.hex())

3031300d060960864801650304020105000420
003031300d060960864801650304020105000420435753d9a6b10bfa294cd7ee950ba60e367075a5db530b129d17f5d10ad1f17b


In [9]:
#This has to be 1 because if the last bit is zero, then it becomes impossible to forge a suffix using this method.
get_bit(from_bytes(suffix),0)

1

In [10]:
new_sig_suffix=1

for i in range(len(suffix)*8):
    if get_bit(new_sig_suffix**3,i) != get_bit(from_bytes(suffix),i):
        new_sig_suffix=set_bit(new_sig_suffix,i)
        #print(bin(start_sig))

In [11]:
len(to_bytes((new_sig_suffix)))

52

In [121]:
to_bytes(new_sig_suffix**3).hex()

'054e6f3c32ed03bd90567a28e2b7dcb842faa47ad7a4a96595c35f3ba10d27c0312c509a9458e7fd4e3640afcadb73b021adba5d892e9de868ae71505d4f0e201282ee21c70bd2d54a86df95e4aa9cb513bb610868620c4d5a5eeb85aee8fbf2529ec411694041b0003031300d060960864801650304020105000420435753d9a6b10bfa294cd7ee950ba60e367075a5db530b129d17f5d10ad1f17b'

In [12]:
to_bytes((new_sig_suffix)**3).endswith(suffix)

True

In [115]:
while True:
    # os.urandom for getting Random Bytes of specific length suitable for cryptography
    new_sig_prefix=b'\x00\x01'+os.urandom(2048//8 - 2)
   
    
    # Generating Prefix and taking the prefix upto the length of suffix
    
    new_sig_prefix=to_bytes(cube_root(from_bytes(new_sig_prefix)))[:-len(suffix)]
    #print(len(to_bytes(from_bytes(new_sig_prefix)**3)))
    
    new_sig=new_sig_prefix+to_bytes(new_sig_suffix)
    
    # We want length to be 85, because 256/3 is almost 85.
    if len(new_sig) > 85:
        new_sig=new_sig_prefix[:-(len(new_sig)-85)]+to_bytes(new_sig_suffix)
    elif len(new_sig) < 85:
        new_sig=new_sig_prefix+'\xFF'*(85-len(new_sig))+to_bytes(new_sig_suffix)
    else:
        new_sig=new_sig_prefix+to_bytes(new_sig_suffix)
        
    #There should be no \x00 in the cube of the signature otherwise verify function will fail 
    if b'\x00' not in to_bytes(from_bytes(new_sig)**3)[2:-len(suffix)]:
        print(new_sig.hex())
        #print(len(to_bytes(from_bytes(new_sig)**3)))
        break

2d5eb31a3fb3025e71b254de71d4ea35793ea6c5c76c57da5c9939488b259aad1846528062dbab8c39ac1d6a14e204a895ea5f847e1b60743ca6580ba86c50099d2227ad25ce0d7f223955679f9199b9ee91fdfe23


In [113]:
#Hex value of the signature seen on the victim side after decryption i.e. sig**3
to_bytes(from_bytes(new_sig)**3).hex()

'0198c9e0b263a0254ff0f5741ac68a97ff5237eb0cf4334a38b3700f47c33581d83ffd70feff310719eb157fe50eebafc1de2e9b39a54a113dc5215b3a0303f2949f6848c5eb58582f929e56466b6f410ad7ee0c14475d9644445235bec44ccddc7bf1fe4616599a3104e3fc03ecbf675b0639a1b0168b151cbe5aebe3c86ed3726e94692a1217ace599e872aa57dc5da1260cbdfeb07f21ee9ceac8918ee5b98527e57dbce2fc7aeb9412abd0615e6109f04e66c62d561e1d6bf6cdb3905f3363f7a215b29d36ab003031300d060960864801650304020105000420435753d9a6b10bfa294cd7ee950ba60e367075a5db530b129d17f5d10ad1f17b'

In [77]:
# Generating new Keys
key=rsa.newkeys(2048)[0]

In [78]:
# Changing public key component to 3
key.e=3

In [109]:
key

PublicKey(20519768567786630875477999578888994751771064050356093104671766931405716494568213155199930484401526140639276710219709466484808649764965004491334674872350505825174731226241640485231932699568272231448520813750527510269968503792555172786733345299893332373364313784056351292044572217348943524141005249442493035840600945669914962750362917689266058728986075610946992139566055707797064590156081130453759370672947127621519871224376502455847713723821242276110209212057509685665078454346228270138016274597182576800091115113780329156890150076744354988599725696215034215937133616626389120918341616363948668950117338075301515105501, 3)

In [80]:
rsa.verify(message,new_sig,key)

True